In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# Set the device for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# MNIST dataset
train_dataset = dsets.MNIST(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

# Data loader
batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Model configuration
input_dim = 28
hidden_dim = 100
layer_dim = 1
output_dim = 10

model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training
total_step = len(train_loader)
num_epochs = 5

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device).view(-1, input_dim, input_dim)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device).view(-1, input_dim, input_dim)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print('Test Accuracy: {:.2f}%'.format(accuracy))


100%|██████████| 9912422/9912422 [00:00<00:00, 80494305.47it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 18262580.10it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 21792493.50it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3894220.93it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/5], Step [100/600], Loss: 2.2903
Epoch [1/5], Step [200/600], Loss: 2.2982
Epoch [1/5], Step [300/600], Loss: 2.2871
Epoch [1/5], Step [400/600], Loss: 2.2881
Epoch [1/5], Step [500/600], Loss: 2.2649
Epoch [1/5], Step [600/600], Loss: 2.2309
Epoch [2/5], Step [100/600], Loss: 2.2055
Epoch [2/5], Step [200/600], Loss: 2.0089
Epoch [2/5], Step [300/600], Loss: 1.5174
Epoch [2/5], Step [400/600], Loss: 1.2025
Epoch [2/5], Step [500/600], Loss: 1.1752
Epoch [2/5], Step [600/600], Loss: 0.8215
Epoch [3/5], Step [100/600], Loss: 0.8550
Epoch [3/5], Step [200/600], Loss: 0.5982
Epoch [3/5], Step [300/600], Loss: 0.6828
Epoch [3/5], Step [400/600], Loss: 0.6916
Epoch [3/5], Step [500/600], Loss: 0.4724
Epoch [3/5], Step [600/600], Loss: 0.4070
Epoch [4/5], Step [100/600], Loss: 0.4541
Epoch [4/5], Step [200/600], Loss: 0.3946
Epoch [4/5], Step [300/600], Loss: 0.2262
Epoch [4/5], Step [400/600], Loss: 0.3341
E